# notebook version of GenerativeExampleEngage.py

In [ ]:
import os

# print(os.getcwd())
import sys

# sys.path.append("../../../")
import torch
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.nn.norm import GraphNorm
from torch_geometric.utils import negative_sampling

from CreateScenes import objs, CreateSceneFromGNNOutput
from Elements.pyGLV.GL.Scene import Scene
import Converter as Converter
import random
import numpy as np
import features.usd.UsdImporterENGAGE as SceneLoader
from Elements.pyECSS.Entity import Entity
from Elements.pyECSS.Component import BasicTransform, RenderMesh, Camera
from CreateScenes import CreateRoomScene,CreateORScene,CreatePaperScene



In [ ]:

# if not os.path.exists("Training_Scenes"):
#     import requests
#     import zipfile

#     # Dropbox link to the zip file
#     dropbox_link = "https://www.dropbox.com/s/nv9lm9jyjwx7tj6/Training_Scenes2.zip?dl=1"

#     # Download the zip file
#     response = requests.get(dropbox_link, stream=True)
#     zip_filename = "file.zip"

#     with open(zip_filename, "wb") as file:
#         for chunk in response.iter_content(chunk_size=128):
#             file.write(chunk)

#     # Extract the contents of the zip file
#     with zipfile.ZipFile(zip_filename, "r") as zip_ref:
#         zip_ref.extractall("Training_Scenes")

#     # Clean up the downloaded zip file
#     import os

#     os.remove(zip_filename)
# else:
#     print("Skipping download, scenes already exist.")


In [ ]:

numscenes = 1000
mydata = []
myy = []
tempobs = objs.copy()
tempobs.remove("root")
label_emb = 8
features = 7
latent_dim = 16


In [ ]:

class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)  # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, 4 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)
        self.norm = GraphNorm(in_channels)
        self.lRelu = torch.nn.LeakyReLU(0.1)

    def forward(self, x, edge_index, batch=None):
        if self.training:
            x = self.norm(x, batch)
        else:
            x = self.norm(x)
        x = self.lRelu(self.conv1(x, edge_index))
        # x = self.lRelu(self.conv2(x, edge_index))
        mu, logstd = self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)
        return self.reparametrize(mu, logstd), mu, logstd

    def reparametrize(self, mu, logstd):
        if self.training:
            return mu + torch.randn_like(logstd) * torch.exp(logstd)
        else:
            return mu



In [ ]:

class VariationalGCNDecoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNDecoder, self).__init__()
        self.lin1 = torch.nn.Linear(in_channels, out_channels)
        self.lin3 = torch.nn.Linear(out_channels, out_channels)
        self.lin2 = torch.nn.Linear(in_channels, in_channels)
        self.lRelu = torch.nn.LeakyReLU(0.1)

    def forward(self, z):
        x = self.lRelu(self.lin1(z))
        x = self.lin3(x)
        # value = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=1)
        return x, z

    def forward_for_loss(self, z, edge_index):
        zhat = self.lin2(z)
        value = (z[edge_index[0]] * zhat[edge_index[1]]).sum(dim=1)
        return torch.sigmoid(value)

    def forward_all(self, z):
        r"""Decodes the latent variables :obj:`z` into a probabilistic dense
        adjacency matrix.

        Args:
            z (torch.Tensor): The latent space :math:`\mathbf{Z}`.
            sigmoid (bool, optional): If set to :obj:`False`, does not apply
                the logistic sigmoid function to the output.
                (default: :obj:`True`)
        """
        zhat = self.lin2(z)
        adj = torch.matmul(z, zhat.t())
        return torch.sigmoid(adj)


In [ ]:

class VariationalAE(torch.nn.Module):
    def __init__(self, encoder, decoder, label_emb):
        super(VariationalAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.__mu__, self.__logstd__ = None, None
        # self.__logstd__ = self.__logstd__.clamp(max=MAX_LOGSTD)
        self.labelEmb = torch.nn.Embedding(5, label_emb) # <-- NOTE: changed first argumnert in emebedding layer!

    def forward(self, x, edge_index, y, batch=None):
        y = self.labelEmb(y) # dim: (bs x 1) x labeL_emb
        # y = y.repeat(x.) # mine: replicate y along each node (x 1st dim is bs X trs_nodes )
        x = torch.cat((x, y), dim=1)
        MAX_LOGSTD = 10
        z, mu, logstd = self.encoder(x, edge_index, batch)
        self.__mu__ = mu
        self.__logstd__ = logstd
        self.__logstd__ = self.__logstd__.clamp(max=MAX_LOGSTD)
        z = torch.cat((z, y), dim=1)
        return self.decoder(z)

    def recon_loss(self, z, pos_edge_index,
                   neg_edge_index=None):
        r"""Given latent variables :obj:`z`, computes the binary cross
        entropy loss for positive edges :obj:`pos_edge_index` and negative
        sampled edges.

        Args:
            z (torch.Tensor): The latent space :math:`\mathbf{Z}`.
            pos_edge_index (torch.Tensor): The positive edges to train against.
            neg_edge_index (torch.Tensor, optional): The negative edges to
                train against. If not given, uses negative sampling to
                calculate negative edges. (default: :obj:`None`)
        """
        EPS = 1e-15
        out = self.decoder.forward_for_loss(z, pos_edge_index)
        pos_loss = -torch.log(out + EPS).mean()

        if neg_edge_index is None:
            neg_edge_index = negative_sampling(pos_edge_index, z.size(0))
        out = self.decoder.forward_for_loss(z, neg_edge_index)
        neg_loss = -torch.log(1 - out + EPS).mean()

        return pos_loss + neg_loss

    def kl_loss(self, mu=None,
                logstd=None):
        r"""Computes the KL loss, either for the passed arguments :obj:`mu`
        and :obj:`logstd`, or based on latent variables from last encoding.

        Args:
            mu (torch.Tensor, optional): The latent space for :math:`\mu`. If
                set to :obj:`None`, uses the last computation of :math:`\mu`.
                (default: :obj:`None`)
            logstd (torch.Tensor, optional): The latent space for
                :math:`\log\sigma`.  If set to :obj:`None`, uses the last
                computation of :math:`\log\sigma^2`. (default: :obj:`None`)
        """
        MAX_LOGSTD = 10
        mu = self.__mu__ if mu is None else mu
        logstd = self.__logstd__ if logstd is None else logstd.clamp(
            max=MAX_LOGSTD)
        return -0.5 * torch.mean(
            torch.sum(1 + 2 * logstd - mu ** 2 - logstd.exp() ** 2, dim=1))

    def add_to_scene(self, additions, data=None, test=False):
        scene = None
        if data is not None and test == False:
            x = data.x
            y = data.y
            edge = data.edge_index
            yInitEmb = self.labelEmb(y)
            xInitEmb = torch.cat((x, yInitEmb), dim=1)
            z, _, _ = self.encoder(xInitEmb, edge)
            z = torch.cat((z, yInitEmb), dim=1)
            additionZ = torch.randn(len(additions), latent_dim).to(device)
            additionsTensor = torch.LongTensor(additions).to(device)
            yAdditionEmb = self.labelEmb(additionsTensor)
            additionZ = torch.cat((additionZ, yAdditionEmb), dim=1)
            newY = torch.cat((y, additionsTensor), dim=0)
            newgraphZ = torch.cat((z, additionZ), dim=0)
            recon = self.decoder(newgraphZ)
            recon = recon[0]
            adj = self.decoder.forward_all(newgraphZ)
            Scene.reset_instance()
            scene = CreateSceneFromGNNOutput(recon, adj, newY, True)
        elif data is not None and test:
            x = data.x
            y = data.y
            allz = None
            for i in range(x.shape[0]):
                tempx = x[i].reshape(1, x.shape[1])
                tempy = y[i].reshape(1, )
                edge = torch.zeros((2, 1)).long().to(device)
                yInitEmb = self.labelEmb(tempy)
                xInitEmb = torch.cat((tempx, yInitEmb), dim=1)
                z, _, _ = self.encoder(xInitEmb, edge)
                z = torch.cat((z, yInitEmb), dim=1)
                if allz is None:
                    allz = z
                else:
                    allz = torch.cat((allz, z), dim=0)
            additionZ = torch.randn(len(additions), latent_dim).to(device)
            additionsTensor = torch.LongTensor(additions).to(device)
            yAdditionEmb = self.labelEmb(additionsTensor)
            additionZ = torch.cat((additionZ, yAdditionEmb), dim=1)
            newY = torch.cat((y, additionsTensor), dim=0)
            newgraphZ = torch.cat((allz, additionZ), dim=0)
            recon = self.decoder(newgraphZ)
            recon = recon[0]
            adj = self.decoder.forward_all(newgraphZ)
            Scene.reset_instance()
            scene = CreateSceneFromGNNOutput(recon, adj, newY, True)
        else:
            additions.append(0)
            additionZ = torch.randn(len(additions), latent_dim).to(device)
            additionsTensor = torch.LongTensor(additions).to(device)
            yAdditionEmb = self.labelEmb(additionsTensor)
            additionZ = torch.cat((additionZ, yAdditionEmb), dim=1)
            newY = additionsTensor
            newgraphZ = additionZ
            recon = self.decoder(newgraphZ)
            recon = recon[0]
            adj = self.decoder.forward_all(newgraphZ)
            Scene.reset_instance()
            scene = CreateSceneFromGNNOutput(recon, adj, newY, True)
        return scene


# Data

In [ ]:
# from os import path
# mystr = path.join('../../Training_Scenes/Training_Scenes2/scene' + str(1) + '.usd')
# mystr

In [ ]:
# ls -lah ../../../Training_Scenes/Training_Scenes2/scene1.usd

In [ ]:
# LOAD 1 scene
Scene.reset_instance()
scene = Scene()
rootEntity = scene.world.createEntity(Entity(name="RooT"))
# SceneLoader.LoadScene(scene, "../../../Training_Scenes/Training_Scenes2/scene" + str(1) + ".usd")
SceneLoader.LoadScene(scene, "./Training_Scenes/Training_Scenes2/scene" + str(1) + ".usd")

In [ ]:
# visualize
# from CreateScenes import view_scene

In [ ]:
# view_scene(scene)    <---- FAILS HERE!

In [ ]:
# # convert 1 scene --> graph
# data = Converter.ECStoGNNSimpleTransQuat(scene)
# data

In [ ]:
# # convert back to ECS
# data_ECS = Converter.GNNtoECS(data)

## create a default room scene and move the chair (ONCE)

In [ ]:
Scene.reset_instance()
scene = CreateRoomScene(visualize=True)

In [ ]:
for comp in scene.world.components:

    if isinstance(comp, BasicTransform):
        if comp.parent.name == 'Chair':  # 'Chair1' # 'Lamp'
            # translate this!
            comp.translation[1] = 0.5 # 1 => Z-axis
            # NOTE: modify BasicTransform as well!!
            comp.trs[1][-1] = 0.5  # trs is 4x4 with the last COL the translation 

In [ ]:
# just verify that is changed
# scene.world.components

# visualize
from CreateScenes import view_scene

In [ ]:
view_scene(scene)

In [ ]:
# N = 5
# for i in range(N):
#     dz = 1./float(N)
#     new_z = float(i) * dz
#     print(new_z)

## Create multiple scenes and move the chair infinitesimally in Z-axis (so that it doesn't collide with other objects...)

In [ ]:
dataGNN_chairs = []
N = 5
for idx in range(N): #(20):
    Scene.reset_instance()
    scene = CreateRoomScene(visualize=False)

    # Z-axis translation
    dz = 1./float(N)
    new_z = float(idx) * dz

    for i, comp in enumerate(scene.world.components):

        if isinstance(comp, BasicTransform):
            if comp.parent.name == 'Chair':  # 'Chair1' # 'Lamp'
                # translate this!
                scene.world.components[i].translation[1] = new_z # 1 => Z-axis
                # NOTE: modify BasicTransform as well!!
                scene.world.components[i].trs[1][-1] =  new_z # trs is 4x4 with the last COL the translation 
                
    # CONVERT to graph
    SceneGNN = Converter.ECStoGNNNoNoise(scene) # NOTE that no noise is used, the only diff is the displaced CHAIR!
    # conditional variable must be a tensor
    y = []
    n_trs_nodes = SceneGNN['trs'].x.shape[0] # number of BasisTrs nodes
    y = [idx] * n_trs_nodes # y.append(idx)
    SceneGNN.t = torch.LongTensor(np.asarray(y)) # time conditional variable. ==> has to be integer

    dataGNN_chairs.append(SceneGNN)

In [ ]:
dummy = []
dummy = [idx] * SceneGNN['trs'].x.shape[0]
dummy

In [ ]:
len(dataGNN_chairs)

In [ ]:
# visualize LAST scene. I cannot save a list of scenes with discrete translations using a variable to modify them.
# view_scene(scene)

In [ ]:
# # Verify that: default Chair trans is: (-0.2, 0, 0)
# Scene.reset_instance()
# def_scene = CreateRoomScene(visualize=False)

# for comp in def_scene.world.components:

#         if isinstance(comp, BasicTransform):
#             if comp.parent.name == 'Chair':  # 'Chair1' # 'Lamp'
#                 # translate this!
#                 print(comp.translation)
#                 print(comp.trs) 

# print('======Modify====')
# # modify and re-print
# for i, comp in enumerate(def_scene.world.components):

#         if isinstance(comp, BasicTransform):
#             if comp.parent.name == 'Chair':  # 'Chair1' # 'Lamp'
#                 # translate this!
#                 def_scene.world.components[i].translation[1] = 0.71
#                 def_scene.world.components[i].trs[1][-1] = 0.71

# print('======print ====')

# for comp in def_scene.world.components:

#         if isinstance(comp, BasicTransform):
#             if comp.parent.name == 'Chair':  # 'Chair1' # 'Lamp'
#                 # translate this!
#                 print(comp.translation)
#                 print(comp.trs) 

In [ ]:
A = np.array([[ 0.1,  0.,   0., -0.2],
 [ 0.,   0.1,  0.,   0. ],
 [ 0.,   0.,   0.1,  0. ],
 [ 0.,   0.,   0.,   1. ]])
print('Default chair trs matrix is=', A)
print('-------')
print('flattened:', A.flatten())

In [ ]:
dataGNN_chairs[0]['trs'].x  # first scene trs matrices. The THIRD corresponds to the chair (default chair). CHECK THAT 8-th position == 0.0

In [ ]:
dataGNN_chairs[1]['trs'].x  # SECOND scene trs matrices. The THIRD corresponds to the chair (MOVED chair in z-axis by 0.2) CHECK THAT 8-th position == 0.2!!

In [ ]:
dataGNN_chairs[2]['trs'].x  # THIRD scene trs matrices. The THIRD corresponds to the chair (MOVED chair in z-axis by 0.4) CHECK THAT 8-th position == 0.4!!

In [ ]:
dataGNN_chairs[2].t # time conditional

In [ ]:
dataGNN_chairs[0]

In [ ]:
dataGNN_chairs[0]['entity', 'trsparent', 'trs']

In [ ]:
dataGNN_chairs[0].t

## attempt to run the cond GVAE

In [ ]:
# re-define parameters
features = 16 # all the trs matrix (flattened)
range_of_embs = 5 # possible discrete items in the embedding space. First argument of torch.nn.Embedding(num_embeddings= , embedding_dim= ) 
label_emb = 3 # let's just use a low one. Second argument of torch.nn.Embedding(num_embeddings= , embedding_dim= )  
latent_dim = 16

In [ ]:
# eachrun = np.zeros((100,2)) #((100, 10))  # epochs x re-runs
# for run in range(2):  # (10):  # re-runs of the same dataset...
#     print("Run:", run)


# random.shuffle(mydata)
mydata = dataGNN_chairs[:5]  # mydata[:100] # [:500]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = VariationalGCNEncoder(features + label_emb, latent_dim)
decoder = VariationalGCNDecoder(latent_dim + label_emb, features)
model = VariationalAE(encoder, decoder, label_emb).to(device)  # TODO: I should use the additional agrument: range_of_embs
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 100
batch_size = 8
train_loader = DataLoader(mydata, batch_size=batch_size, shuffle=True)
criterion = torch.nn.MSELoss()
model.train()
allrows = []
alllosses = []

for i in range(epochs):
    losses = []
    for t in train_loader:
        t = t.to(device)
        optimizer.zero_grad()
        x, z = model(t['trs'].x, t['entity', 'trsparent', 'trs'].edge_index, t.t, t['trs'].batch) # model(t.x, t.edge_index, t.y, t.batch)
        loss1 = model.recon_loss(z, t.edge_index)  # reconstruction loss on edges
        loss2 = criterion(x, t.x)                  # reconstruction loss on node embeds
        loss3 = (1 / t.num_nodes) * model.kl_loss()  # Regularization loss on latent space (why does it have a mean in kl_loss() definition?)
        loss = loss1 + loss2 + loss3            # <-- TODO: add weights as in beta-VAE
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print("Epoch ", i, ": Loss:", np.mean(np.asarray(losses)))
    # eachrun[i][run] = np.mean(np.asarray(losses))
    # row = ["Epoch " + str(i), str(np.mean(np.asarray(losses)))]
    # allrows.append(row)
    # alllosses.append(np.mean(np.asarray(losses)))

In [ ]:
mydata[0].t

## Load usd data to Scenes (paper Section 5.2)

In [ ]:
mydata = []
for i in range(2): # (1000):
    Scene.reset_instance()
    scene = Scene()
    rootEntity = scene.world.createEntity(Entity(name="RooT"))
    SceneLoader.LoadScene(scene, "./Training_Scenes/Training_Scenes2/scene" + str(i) + ".usd")
    # This line can be changed to different GA converting functions, for example TRS, CGA etc.
    data = Converter.ECStoGNNSimpleTransQuat(scene)
    print('i', i)
    mydata.append(data)  # MINE <----------!!!!!!!!!!!!11


In [ ]:
print(mydata[0].x)
print('----')
print(mydata[0].y)
print('----')
print(mydata[0].edge_index)

print('====== 2nd ======')

print(mydata[1].x)
print('----')
print(mydata[1].y)
print('----')
print(mydata[1].edge_index)


In [ ]:
# clear memory
# mydata =[]  # does not free RAM
# mydata.clear() # does NOT free RAM, only contents

In [ ]:
print(mydata[0])
print("=======")
print('labels= ', mydata[0].y) # NOTE: this varies from scene to scene!

In [ ]:
# print parameters set at the beggining
print('label_emb =', label_emb) #  why??
print('features =', features) # 2nd dim of data.x
print('latent_dim =', latent_dim) 

In [ ]:
len(mydata)

In [ ]:
# import gc 

# print(gc.get_threshold())

# del(mydata) # should work if there are NO references to 'mydata'... Didn't work for me... PROBABLY due to last 'data' living in CUDA
# gc.collect()

In [ ]:
eachrun = np.zeros((100,2)) #((100, 10))  # epochs x re-runs
for run in range(2):  # (10):  # re-runs of the same dataset...
    print("Run:", run)
    random.shuffle(mydata)
    mydata = mydata[:100] # [:500]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    encoder = VariationalGCNEncoder(features + label_emb, latent_dim)
    decoder = VariationalGCNDecoder(latent_dim + label_emb, features)
    model = VariationalAE(encoder, decoder, label_emb).to(device)   
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    epochs = 100
    batch_size = 8
    train_loader = DataLoader(mydata, batch_size=batch_size, shuffle=True)
    criterion = torch.nn.MSELoss()
    model.train()
    allrows = []
    alllosses = []
    for i in range(epochs):
        losses = []
        for t in train_loader:
            t = t.to(device)
            optimizer.zero_grad()
            x, z = model(t.x, t.edge_index, t.y, t.batch)
            loss1 = model.recon_loss(z, t.edge_index)
            loss2 = criterion(x, t.x)
            loss3 = (1 / t.num_nodes) * model.kl_loss()  # new line
            loss = loss1 + loss2 + loss3
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
        print("Epoch ", i, ": Loss:", np.mean(np.asarray(losses)))
        eachrun[i][run] = np.mean(np.asarray(losses))
        row = ["Epoch " + str(i), str(np.mean(np.asarray(losses)))]
        allrows.append(row)
        alllosses.append(np.mean(np.asarray(losses)))

In [ ]:
device